# Networks: AMP-PD PPMI x PDBP - January 2021
 - **Project:** Build networks based on underlying data and Mary's feature importance.
 - **Author(s):** Mary Makarious, Dan Vitale, Mike Nalls 

---
### Quick Description: 
- **Problem:** These genetic features are important, are then new networks / communities? 
- **Solution:** The list below sums it up well ...  

1.   Pull genes - this includes querying QTLs from open targets API in brain tissues to connect SNPs to genes.
2.   Extract from AMP data.
3.   Build correlation / graph space in cases.
4.   Leiden to build communties.
5.   Vizualize.




# Imports and set up.

In [1]:
# Imports and set up of the colab.
import os
from google.colab import drive
import pandas.util.testing as tm
import h5py
import numpy as np
import pandas as pd
import math
import sys
import joblib
import subprocess
import statsmodels.api as sm
from scipy import stats

! pip install --upgrade tables
import tables

drive.mount('/content/drive/')
os.chdir("/content/drive/Shared drives/LNG/users/makariousmb/projects/PPMIxPDBP_GenoML/mike_networks")
! pwd

import requests
import pandas as pd
import json

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  after removing the cwd from sys.path.


     |████████████████████████████████| 4.3MB 5.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
Mounted at /content/drive/
/content/drive/Shared drives/LNG/users/makariousmb/projects/PPMIxPDBP_GenoML/mike_networks


# Pull the genes of interest from the feature importance file.

Read in features and extract the 'ENSG' to one list and rsIDs to another.

In [2]:
# Quick and dirty styles.
! grep 'ENSG' validation_PDBP.finalHarmonizedCols_toKeep.txt > rna_from_impMat.txt
! grep 'rs' validation_PDBP.finalHarmonizedCols_toKeep.txt > snps_from_impMat.txt

In [ ]:
# API calls set up.

def search_rsid_open_targets_query(rsid):

  """
  query open targets for snp ids in chr_pos_a1_a2 and allele freqs with rsid

  input: rsid
  output: ids and allele frequencies for different ancestry groups with columns:
  id	rsId	gnomadAFR	gnomadAMR	gnomadASJ	gnomadEAS	gnomadFIN	gnomadNFE	
    gnomadNFEEST	gnomadNFENWE	gnomadNFESEU	gnomadNFEONF	gnomadOTH

  Some information on the allele frequencies being pulled 

  "gnomAD Allele frequency (African/African-American population)"
    gnomadAFR: Float

    "gnomAD Allele frequency (Latino/Admixed American population)"
    gnomadAMR: Float

    "gnomAD Allele frequency (Ashkenazi Jewish population)"
    gnomadASJ: Float

    "gnomAD Allele frequency (East Asian population)"
    gnomadEAS: Float

    "gnomAD Allele frequency (Finnish population)"
    gnomadFIN: Float

    "gnomAD Allele frequency (Non-Finnish European population)"
    gnomadNFE: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Estonian sub-population)"
    gnomadNFEEST: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean North-Western European sub-population)"
    gnomadNFENWE: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Southern European sub-population)"
    gnomadNFESEU: Float

    "gnomAD Allele frequency (Non-Finnish Eurpoean Other non-Finnish European sub-population)"
    gnomadNFEONF: Float

    "gnomAD Allele frequency (Other (population not assigned) population)"
    gnomadOTH: Float
    """

  api_query = """
  query search($queryString: String!){
    search(queryString: $queryString) {
      variants{
        id
        rsId
        gnomadAFR
        gnomadAMR
        gnomadASJ
        gnomadEAS
        gnomadFIN
        gnomadNFE
        gnomadNFEEST
        gnomadNFENWE
        gnomadNFESEU
        gnomadNFEONF
        gnomadOTH
      }
    }
  }"""

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"
  #set variables object
  variables = {"queryString": rsid}

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query':api_query, "variables":variables})
  if str(r.status_code) == '400':
    print(f'{rsid} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json

def query_qtls(snp_list):
  """
  uses qtl_coloc_open_targets_query() function to query open targets for qtls
    given a list of snps in chr_pos_a1_a2 format (hg38)

  input: list of snps (chr_pos_a1_a2 formatted with hg38 positions)
  output: cleaned up dataframe of qtls with following columns:
  snp	gene_symbol	gene_id	type	tissue	beta	pval
  """

  total_qtls_df = pd.DataFrame()
  for snp in snp_list:
    qtl_query = qtl_coloc_open_targets_query(snp)
    qtl_query_df = pd.json_normalize(qtl_query['data']['genesForVariant'])
    if len(qtl_query_df) !=  0:
      qtl_query_df['snp'] = snp
      qtls_df = qtl_query_df.loc[qtl_query_df.qtls.map(lambda d: len(d)) > 0].reset_index(drop=True).copy()
      total_qtls_df = total_qtls_df.append(qtls_df)
  total_qtls_df = total_qtls_df.reset_index(drop=True)

  final_qtls_df = pd.DataFrame()
  for i, qtl in enumerate(total_qtls_df.qtls):
    for j, qt in enumerate(qtl):
      for k, tissue in enumerate(qt['tissues']):

        qtl_dict = {
            'snp': total_qtls_df.loc[i,'snp'],
            'gene_symbol': total_qtls_df.loc[i,'gene.symbol'],
            'gene_id': total_qtls_df.loc[i,'gene.id'],
            'type': qt['typeId'],
            'tissue': tissue['tissue']['name'],
            'beta': tissue['beta'],
            'pval': tissue['pval']
        }
        
        qtl_dict_df = pd.DataFrame.from_records([qtl_dict])
        final_qtls_df = final_qtls_df.append(qtl_dict_df).reset_index(drop=True)
        
  return final_qtls_df

  
def qtl_coloc_open_targets_query(variant_id):
  """
  queries open targets for qtl data given variant_id in chr_pos_a1_a2 format 
    (hg38)

  input: variant id in chr_pos_a1_a2 format (hg38)
  output: dataframe with the following columns:
  qtls gene.symbol gene.id
  """
  api_query = f'''query {{
    genesForVariant(variantId: "{variant_id}") {{
      gene {{
        id
        symbol
      }}
      qtls{{
        typeId
        sourceId
        aggregatedScore
        tissues{{
          tissue {{
            id
            name
          }}
          quantile
          beta
          pval
        }}
      }}
    }}
  }}'''

  #set base_url for Open Targets Genetics Portal API
  base_url = "http://genetics-api.opentargets.io/graphql"

  #perform API call and check status code of response
  r = requests.post(base_url, json={'query': api_query})
  if str(r.status_code) == '400':
    print(f'{variant_id} query status code: {r.status_code}')
  
  else:
    pass

  #transform API response into JSON 
  api_response_as_json = json.loads(r.text)
  
  return api_response_as_json



Query open targets API. Go from rsID to gr38 ID to filtered gene IDs taken from brain QTLs.

In [ ]:
# Get the variant style IDs as opposed to the rsIDs to query open targets API.

SNP_list_df = pd.read_csv("snps_from_impMat.txt", names=['SNP'])
SNP_list = SNP_list_df['SNP'].unique()

these_SNPs = []

for i in range(len(SNP_list)):
  this_SNP = SNP_list[i]
  rsid_query = search_rsid_open_targets_query(this_SNP)
  rsid_query_df = pd.json_normalize(rsid_query['data']['search']['variants'])
  if hasattr(rsid_query_df, 'id'):
    this_variant_stringed = str(rsid_query_df['id'])
    this_variant = this_variant_stringed.split()[1]
  else:
    this_variant = np.nan
  this_variant = this_variant_stringed.split()[1]
  print(this_SNP, this_variant)
  these_SNPs.append((this_SNP, this_variant))

these_SNPs_df = pd.DataFrame(these_SNPs, columns=('SNP', 'VARIANT'))
these_SNPs_df.to_csv("variants_mined_01072021.csv", index=False)


In [ ]:
# Now get the gene IDs for QTLs in brain tissues relating to these SNPs.

these_SNPs_df = pd.read_csv("variants_mined_01072021.csv")
variant_list = these_SNPs_df['VARIANT'].unique()

qtls_df = query_qtls(variant_list)
qtls_df.head()

qtls_df.to_csv("QTLS_mined_01072021.csv", index=False)


In [ ]:
# Now subset that QTL list to tissues of interest.

Merge the two gene lists and make unique.

In [ ]:
qtls_df = pd.read_csv("QTLS_mined_01072021.csv", engine='c')
rna_df = pd.read_csv("rna_from_impMat.txt", engine='c', names=['gene_id'])

In [ ]:
qtls_df.head()
# qtls_df.describe()

,snp,gene_symbol,gene_id,type,tissue,beta,pval
0,4_47509764_T_C,ATP10D,ENSG00000145246,eqtl,Lcl (TWINSUK),0.090071,7.271000e-07
1,4_47509764_T_C,ATP10D,ENSG00000145246,eqtl,Lcl (GENCORD),0.154816,5.169660e-06
2,4_47509764_T_C,NFXL1,ENSG00000170448,eqtl,Blood (eQTLGen),0.076645,1.287900e-19
3,18_49054943_G_A,DYM,ENSG00000141627,eqtl,Blood (eQTLGen),0.165194,9.361700e-79
4,4_25814426_A_G,SEL1L3,ENSG00000091490,eqtl,Monocyte naive (FAIRFAX 2014),-0.409197,4.979690e-06


In [ ]:
rna_df.head()
# rna_df.describe()

,gene_id
0,ENSG00000000938
1,ENSG00000002919
2,ENSG00000003509
3,ENSG00000004478
4,ENSG00000004948


In [ ]:
qtl_gene_ids = qtls_df['gene_id'].unique()
rna_gene_ids = rna_df['gene_id'].unique()
gene_ids_to_pull = np.concatenate([qtl_gene_ids, rna_gene_ids])
unique_gene_ids_to_pull = pd.unique(gene_ids_to_pull)

9797

Pull these genes from gene expression data.

In [ ]:
# Load in and reduce the working data.
raw_df = pd.read_hdf("../mike_diff_exp/data_for_diff_exp.h5", key='diff_exp_df', mode='r')
case_df = raw_df[raw_df['PHENO'] == 1]
data_df = case_df[unique_gene_ids_to_pull]

# Make the correlation matrix / graph space in cases only.
Then filter it at 0.8 positve correlation.

In [ ]:
# Quick imports.
import networkx as nx
import community
from cdlib import algorithms, readwrite
# ! pip install cdlib

In [ ]:
# Correlate the variables.
cor_matrix = data_df.iloc[:,1:].corr()
links = cor_matrix.stack().reset_index()
links.columns = ['feature1', 'feature2','corr']

In [ ]:
# filter at positve correlation > 0.8.
links_filtered = links.loc[ (links['corr'] >= 0.8) & (links['feature1'] != links['feature2']) ]

In [ ]:
# Make the networkx graph.
G=nx.from_pandas_edgelist(links_filtered, 'feature1', 'feature2')

# Leiden communities.

In [ ]:
coms = algorithms.leiden(G)
readwrite.write_community_csv(coms, "test_coms.csv")

In [ ]:
coms_df = pd.DataFrame.from_dict(coms, orient='index')

TypeError: ignored

Quick page ranking.

In [ ]:
pageranked = nx.pagerank(G)
pageranked_df = pd.DataFrame.from_dict(pageranked, orient='index')
pageranked_df.reset_index(level=0, inplace=True)
pageranked_df.rename(columns={"index":"feature_name",0:"page_rank"}, inplace=True)
pageranked_outfile = 'network_pageranks.csv'
pageranked_df.to_csv(partition_outfile, index=False)
pageranked_df.head()


NameError: ignored

# A pretty picture to visualize and export the communities.